In [4]:
# import libraries
import time

import pandas as pd
import numpy as np
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By

In [1]:
url = "https://www.vrbo.com/search?adults=2&children=&regionId=122251&destination=Kitty%20Hawk%2C%20North%20Carolina%2C%20United%20States%20of%20America&privacyTrackingState=CAN_TRACK&searchId=033135fc-c70d-4793-aab6-8c1a158f2030&sort=RECOMMENDED&useRewards=SHOP_WITHOUT_POINTS&theme=&pwaDialog=&semdtl=&isInvalidatedDate=false&userIntent="

In [2]:
from vrbo_tool import main

df = main(url)

This is the page number:  1
This is the page number:  2
This is the page number:  3
This is the page number:  4
Finished collecting ulrs, collected 147


ValueError: All arrays must be of the same length

In [13]:
df = pd.read_csv("vrbo_links_old.csv")
df = df.iloc[9:, 1:]
df = df.reset_index(drop=True)
df

,links
0,https://www.vrbo.com/2632787?dateless=true&x_p...
1,https://www.vrbo.com/9864882ha?dateless=true&x...
2,https://www.vrbo.com/3325136?dateless=true&x_p...
3,https://www.vrbo.com/3195090?dateless=true&x_p...
4,https://www.vrbo.com/1719413?dateless=true&x_p...
...,...
468,https://www.vrbo.com/9908530ha?dateless=true&x...
469,https://www.vrbo.com/9379928ha?dateless=true&x...
470,https://www.vrbo.com/9798903ha?dateless=true&x...
471,https://www.vrbo.com/3406794?dateless=true&x_p...


In [15]:
driver = Chrome()

In [ ]:
# Define lists
urls_list = []
titels_list = []
prices_list = []

bedrooms_list = []
bathrooms_list = []
sleeps_list = []

popular_amenitie_0_list = []
popular_amenitie_1_list = []
popular_amenitie_2_list = []
popular_amenitie_3_list = []
popular_amenitie_4_list = []
popular_amenitie_5_list = []

# first drop un nessacry links
urls = df
url_num = 1

for index in range(len(urls)):  # Iterate over all links
    url = urls.iloc[index].values[0]
    urls_list.append(url)  # Add the url to the data_frame

    driver.get(url)
    time.sleep(3)

    try:
        title = driver.find_elements(
            By.XPATH, "//*[@class = 'uitk-heading uitk-heading-3']"
        )[0].text
        titels_list.append(title)  # The title of the listing
    except:
        titels_list.append(np.nan)

    try:
        price = driver.find_element(By.XPATH, "//*[@class = 'uitk-lockup-price']").text
        prices_list.append(price)  # The price of the listing
    except:
        prices_list.append(np.nan)  # Error handling

    # Items list
    items_3 = driver.find_elements(
        By.XPATH,
        "//*[@class = 'uitk-text uitk-text-spacing-three uitk-type-300 uitk-text-standard-theme uitk-layout-flex-item uitk-layout-flex-item-flex-grow-1']",
    )

    items_4 = driver.find_elements(
        By.XPATH,
        "//*[@class = 'uitk-text uitk-text-spacing-three uitk-type-300 uitk-text-standard-theme uitk-layout-flex-item uitk-layout-flex-item-flex-basis-half_width uitk-layout-flex-item-flex-grow-1']",
    )

    if len(items_3) > 0:  # First if it has three items
        try:
            num_bedrooms = int(items_3[0].text.split()[0])
            bedrooms_list.append(num_bedrooms)
        except:
            bedrooms_list.append(np.nan)

        try:
            num_bathrooms = int(items_3[1].text.split()[0])
            bathrooms_list.append(num_bathrooms)
        except:
            bathrooms_list.append(np.nan)

        try:
            num_sleeps = int(items_3[2].text.split()[1])
            sleeps_list.append(num_sleeps)
        except:
            sleeps_list.append(np.nan)

    elif len(items_4) > 0:
        try:
            num_bedrooms = int(items_4[0].text.split()[0])
            bedrooms_list.append(num_bedrooms)
        except:
            bedrooms_list.append(np.nan)

        try:
            num_bathrooms = int(items_4[1].text.split()[0])
            bathrooms_list.append(num_bathrooms)
        except:
            bathrooms_list.append(np.nan)

        try:
            num_sleeps = int(items_4[2].text.split()[1])
            sleeps_list.append(num_sleeps)
        except:
            sleeps_list.append(np.nan)

    else:
        bedrooms_list.append(np.nan)
        bathrooms_list.append(np.nan)
        sleeps_list.append(np.nan)

    # Popular amenities
    try:
        # All the popular amenities in the listing
        popular_amenities = driver.find_elements(
            By.XPATH,
            "//*[@class = 'uitk-text uitk-type-300 uitk-text-default-theme uitk-layout-flex-item']",
        )

        for i in range(6):
            try:
                globals()[f"popular_amenitie_{i}"] = popular_amenities[i].text

            except:  # If there is less than 6 items
                globals()[f"popular_amenitie_{i}"] = np.nan

        # Add them to lists
        popular_amenitie_0_list.append(popular_amenitie_0)
        popular_amenitie_1_list.append(popular_amenitie_1)
        popular_amenitie_2_list.append(popular_amenitie_2)
        popular_amenitie_3_list.append(popular_amenitie_3)
        popular_amenitie_4_list.append(popular_amenitie_4)
        popular_amenitie_5_list.append(popular_amenitie_5)

    except:  # Error handling if no popular amenities at all
        popular_amenitie_0_list.append(np.nan)
        popular_amenitie_1_list.append(np.nan)
        popular_amenitie_2_list.append(np.nan)
        popular_amenitie_3_list.append(np.nan)
        popular_amenitie_4_list.append(np.nan)
        popular_amenitie_5_list.append(np.nan)

# Create the dataframe
all_data_df = pd.DataFrame(
    {
        "urls": urls_list,
        "title": titels_list,
        "prices": prices_list,
        "num_bedrooms": bedrooms_list,
        "num_bathrooms": bathrooms_list,
        "num_sleep": sleeps_list,
        "popular_amenitie_0": popular_amenitie_0_list,
        "popular_amenitie_1": popular_amenitie_1_list,
        "popular_amenitie_2": popular_amenitie_2_list,
        "popular_amenitie_3": popular_amenitie_3_list,
        "popular_amenitie_4": popular_amenitie_4_list,
        "popular_amenitie_5": popular_amenitie_5_list,
    }
)

IndexError: list index out of range

In [31]:
urls_list[-2]

'https://www.vrbo.com/1024234?dateless=true&x_pwa=1&rfrr=HSR&pwa_ts=1738660416255&referrerUrl=aHR0cHM6Ly93d3cudnJiby5jb20vSG90ZWwtU2VhcmNo&useRewards=false&adults=2&regionId=235&destination=North+Carolina%2C+United+States+of+America&destType=MARKET&latLong=35.719758%2C-79.453125&privacyTrackingState=CAN_TRACK&searchId=a18e4390-4f7a-4891-88cf-a8498957daa4&sort=RECOMMENDED&gclid=CjwKCAiA74G9BhAEEiwA8kNfpQI3jfqFv5rIIHkaPzgVrlALK5ybAW4IImA5xRUpnmTkYFzWLtfjJxoCygwQAvD_BwE&semcid=VRBO-US.B.GOOGLE.BD-c-EN.VR&semdtl=a118921852681.b1145236417018.g1kwd-522436780534.e1c.m1CjwKCAiA74G9BhAEEiwA8kNfpQI3jfqFv5rIIHkaPzgVrlALK5ybAW4IImA5xRUpnmTkYFzWLtfjJxoCygwQAvD_BwE.r12e213678c473a12ebda4545133f2c07ecb5fc68c81df9a7bed9eff41fcdc4ffc.c1ioSXP-1MdtLmUk2KclVEuw.j19073655.k121160.d1639955023346.h1p.i1.l1.n1.o1.p1.q1.s1.t1.x1.f1.u1.v1.w1&semdtl=a118921852681.b1145236417018.g1kwd-522436780534.e1c.m1CjwKCAiA74G9BhAEEiwA8kNfpQI3jfqFv5rIIHkaPzgVrlALK5ybAW4IImA5xRUpnmTkYFzWLtfjJxoCygwQAvD_BwE.r12e213678c473a12eb

In [29]:
len(prices_list)

122

In [25]:
all_data_df

,urls,title,prices,num_bedrooms,num_bathrooms,num_sleep,popular_amenitie_0,popular_amenitie_1,popular_amenitie_2,popular_amenitie_3,popular_amenitie_4,popular_amenitie_5
0,https://www.vrbo.com/903434?dateless=true&x_pw...,HIGH MOUNTAIN CONDO--PANORAMIC VIEWS!,$91,1.0,1.0,4.0,Parking available,Air conditioning,Kitchen,Dryer,Fireplace,Washer
1,https://www.vrbo.com/pdp/lo/43991416?dateless=...,Maggie Valley Cabin Rentals,NaN,NaN,NaN,NaN,Pet friendly,Air conditioning,Kitchen,Barbecue grill,Outdoor Space,Parking included
2,https://www.vrbo.com/3323836?dateless=true&x_p...,Oceanfront End Unit Condo with Pool (Riggings D2),$200,1.0,1.0,4.0,Parking available,Pool,Washer,Barbecue grill,Air conditioning,Kitchen
3,https://www.vrbo.com/3450633?dateless=true&x_p...,FALL SALE | The Cottage- Walk2 Main St - Pkg,$137,2.0,1.0,4.0,Parking available,Air conditioning,Kitchen,Fireplace,Outdoor Space,NaN
4,https://www.vrbo.com/3577672?dateless=true&x_p...,"West Asheville Pet Friendly Sleeps 6, Only 15 ...",$159,2.0,NaN,6.0,Parking available,Pet friendly,Washer,Barbecue grill,Air conditioning,Kitchen
5,https://www.vrbo.com/2769908?dateless=true&x_p...,Little Log Cabin: Cozy 1BR w/ Long Range Views,$160,1.0,1.0,4.0,Parking available,Washer,Air conditioning,Kitchen,Dryer,Outdoor Space
6,https://www.vrbo.com/9871371ha?dateless=true&x...,Paradise By The Sea at Villas of Hatteras Land...,$142,1.0,1.0,4.0,Parking available,Air conditioning,Kitchen,Pool,Outdoor Space,Barbecue grill
7,https://www.vrbo.com/3400062?dateless=true&x_p...,Oceanfront condo with pool. Easy walk to CB Bo...,$250,2.0,2.0,5.0,Parking available,Washer,Air conditioning,Kitchen,Dryer,Outdoor Space
8,https://www.vrbo.com/2823558?dateless=true&x_p...,Country home on an acre!,$116,2.0,1.0,4.0,Parking available,Air conditioning,Kitchen,Dryer,Washer,Outdoor Space
9,https://www.vrbo.com/9819914ha?dateless=true&x...,Beautiful Murphy Cabin w/ Mountain View & Fire...,$102,2.0,2.0,4.0,Parking available,Washer,Air conditioning,Kitchen,Dryer,Fireplace


In [23]:
len(sleeps_list)

10

In [16]:
from vrbo_tool import scrape_urls

scrape_urls(df, driver)

ValueError: All arrays must be of the same length